In [21]:
import os
import pandas as pd
import socket as c
import pickle

In [22]:
#client_IP_address = c.gethostbyname(c.gethostname)
client_IP_address = '130.85.239.232'
client_port = 8088
client_buffer = 1024
encoding_format = 'utf-8'

In [23]:
validation = pd.DataFrame(pd.read_excel("validation.xlsx"))

#data.to_excel("validation.xlsx", index_label="label", merge_cells=False)

print(validation)

# username = input()
# password = input()
# # print()
# for i in range(len(validation["username"])):
#     if validation.loc[i][0] == username:
#         if validation.loc[i][1] == password:
#             print("Correct")
#         else:
#             print("wrong password")
#         break
# else:
#     print(username, " not present")

  username        password
0    dhruv  dhruvjpatel68!
1  payodhi         payodhi
2  dharani         dharani
3    surya           surya


In [24]:
#client_IP_address = sys.argv[1]
#client_port = sys.argv[2]

global username, available_files, dirname

username = "dhruv"

def create_client():
    with c.socket(c.AF_INET, c.SOCK_STREAM) as client:
        client.connect((client_IP_address, client_port))
        #print(client.recv(client_buffer).decode('utf-8'))
        #client.sendall(b"Hello from client")
        
        

#########################################################################
#  Runing the while loop untill the user selects the logout function.
#########################################################################
        while True:
            print("hello")
            

######################################################################                    
#   Implementation of the CRUD operations.
######################################################################


            def createDirectory():
                global username, dirname
                print("\nPlease enter directory name")
                dirname = input()

                features = []
                features.append("createdir")
                features.append(username)
                features.append(dirname)
                print(features)
                features = pickle.dumps(features)
                client.send(features)
                #client.send(features.encode())
                data = client.recv(client_buffer)
                data = data.decode()

                #client.send(features.encode())
                client.send(features)
                data = client.recv(client_buffer)
                data = data.decode()
                print(data+"\n")


            def createFile():

                global username, dirname
                print("\nPlease enter directory name to create file")
                dirname = input()
                print("Please enter file name")
                filename = input()

                features = []
                features.append("createfile")
                features.append(username)
                features.append(dirname)
                features.append(filename)
                #features = pickle.dumps(features)
                client.send(features.encode())
                data = client.recv(100)
                data = data.decode()
                available_files.append('files/'+username+"/"+dirname+"/"+filename)

                client = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
                client.connect(('localhost', 3333))
                client.send(features.encode())
                print(data+"\n")                    


            def writeFile():

                global username, text
                global dirname
                print("\nPlease enter directory name to write data")
                dirname = input()
                print("Please enter file name")
                filename = input()
                print("Please enter file content")
                filetext = input()

                features = []
                features.append("writefile")
                features.append(username)
                features.append(dirname)
                features.append(filename)
                features.append(filetext)
                features = pickle.dumps(features)
                client.send(features.encode())
                data = client.recv(100)
                data = data.decode()
                client.send(features.encode())
                print(data+"\n")


            def readFile():
                global username, dirname

                print("\nPlease enter directory name to read file")
                dirname = input()
                print("Please enter file name to read it")
                filename = input()

                features = []
                features.append("readfile")
                features.append(username)
                features.append(dirname)
                features.append(filename)
                features = pickle.dumps(features)
                client.send(features.encode())
                rec = client.recv(10000)
                dataset = pickle.loads(rec)
                request = dataset[0]
                if request == "correct":
                    data = dataset[1]
                    print("File Content Showing in Below lines\n\n")
                    print(data)
                    print("File Contenet ended here=================================")
                else:
                    print("Given file does not exists\n")

            def deleteFile():

                global username, dirname
                print("\nPlease enter directory name to delete a file")
                dirname = input()
                print("Please enter file name to delete it")
                filename = input()

                features = []
                features.append("deletefile")
                features.append(username)
                features.append(dirname+"/"+filename)
                features = pickle.dumps(features)
                client.send(features.encode())
                data = client.recv(100)
                data = data.decode()
                if data == 'Given file deleted':
                    for i in range(len(available_files)):
                        names = available_files[i]
                        names = names.replace("\\","/")
                        arr = names.split("/")
                        if arr[3] == filename:
                            del available_files[i]
                print(data+"\n")


            def renameFile():

                global username, text
                global dirname

                print("\nPlease enter directory name to rename a file")
                dirname = input()
                print("Please enter old file name for renaming")
                oldname = input()
                print("Please enter new file name to rename")
                newname = input()

                features = []
                features.append("renamefile")
                features.append(username)
                features.append(dirname)
                features.append(oldname)
                features.append(newname)
                features = pickle.dumps(features)
                client.send(features.encode())
                data = client.recv(100)
                data = data.decode()
                for i in range(len(available_files)):
                    names = available_files[i]
                    names = names.replace("\\","/")
                    arr = names.split("/")
                    if arr[3] == oldname:
                        del available_files[i]
                available_files.append('files/'+username+"/"+dirname+"/"+newname)        
                print(data+"\n")


            def listFiles():
                print("\nAvailable files for this user: "+username+"\n\n")
                for i in range(len(available_files)):
                    print(available_files[i])
                print()

            def readFiles():
                global username, available_files
                if len(available_files) > 0:
                    available_files.clear()
                features = []
                features.append("listfiles")
                features.append(username)
                features = pickle.dumps(features)
                client.send(features.encode())
                data = client.recv(10000)
                data = pickle.loads(data)
                for i in range(len(data)):
                    available_files.append(data[i])



###############################################################################                    
# The below are the operations that client can select and implement in server. 
###############################################################################

            print("Enter the operation to be performed")
            data = input("=> ")
            data = data.split(" ")
            operation = data[0]

            if operation.lower().strip() == "help":
                client.send(operation.encode())

            elif operation.lower().strip() == "createdir":
                createDirectory()

            elif operation.lower().strip() == "createfile":
                createFile()

            elif operation.lower().strip() == "writefile":
                writeFile()

            elif operation.lower().strip() == "readfile":
                readFile()

            elif operation.lower().strip() == "deletefile":
                deleteFile()

            elif operation.lower().strip() == "renamefile":
                renameFile()

            elif operation.lower().strip() == "listfile":
                listFiles()

            elif operation.lower().strip() == "create":
                client.send(operation.encode())

            elif operation.lower().strip() == "read":
                client.send(operation.encode())

            elif operation.lower().strip() == "logout":
                client.send(operation.encode())
                break
                
                
                
                
                
            data = client.recv(client_buffer).decode()
            operation, msg = data.split("@")
            print(operation, msg)
            if operation.lower().strip() == "disconnect":
                print(f"{msg}")
                print(f"Disconnected from the server")
                client.close()
                break
            elif operation.lower().strip() == "ack":
                print(f"{msg}")
            elif operation.lower().strip() == "create":
                print(f"{msg}")
                f_name = input("=> ")
                f_name = f_name.split(" ")
                operation = f_name[0]
                client.send(operation.encode())



#             print("Username")
#             username = input()              

In [25]:
if __name__ == "__main__":
    create_client()

hello
Enter the operation to be performed
=> createdir

Please enter directory name
Dhruv
['createdir', 'dhruv', 'Dhruv']


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
f = open("demofile2.txt", "a")
f.write("Now the file has more content!")
f.close()